In [6]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_study_date(driver, url):
    try:
        driver.get(url)
        time.sleep(2)  # let page load

        label_divs = driver.find_elements(By.CLASS_NAME, "MuiGrid-item")

        for idx, div in enumerate(label_divs):
            try:
                label = div.find_element(By.TAG_NAME, "span").text.strip()
                if label == "Study Date:":
                    # Get the next sibling div and extract its span content
                    date_div = label_divs[idx + 1]
                    date_text = date_div.find_element(By.TAG_NAME, "span").text.strip()
                    return date_text
            except Exception:
                continue

        return None  # Not found

    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None


In [7]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time

options = Options()
options.add_argument("--headless")

service = Service(GeckoDriverManager().install())
driver = webdriver.Firefox(service=service, options=options)

# Replace with your test URL
test_url = "https://doi.org/10.25940/ROPER-31086629"
print(get_study_date(driver, test_url))

driver.quit()


July 28, 1983 - August 1, 1983


In [16]:
import pandas as pd
import time
import os
from tqdm import tqdm
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from urllib.parse import urljoin

# --- Configuration ---
INPUT_FILE = "unique_roper_links.csv"         # Replace with your .dta or .csv
OUTPUT_FILE = "roper_scraped.csv"
ERROR_FILE = "scrape_errors.csv"
SAVE_EVERY = 20
MAX_RETRIES = 3
BASE_URL = "https://doi.org/10.29540/ROPER-"
WAIT_SECONDS = 2  # Adjust if needed

# --- Setup headless Firefox driver ---
options = Options()
options.add_argument('--headless')
driver = webdriver.Firefox(options=options)

# --- Load your data ---
df = pd.read_csv(INPUT_FILE)  # or pd.read_csv()
df = df.dropna(subset=['RoperDOI'])

# --- Setup result and error tracking ---
results = []
errors = []

# Resume from existing output
if os.path.exists(OUTPUT_FILE):
    existing = pd.read_csv(OUTPUT_FILE)
    done_ids = set(existing['RoperDOI'])
    results.extend(existing.to_dict(orient='records'))
else:
    done_ids = set()

# --- Scraping loop ---
for _, row in tqdm(df.iterrows(), total=len(df)):
    roper_id = row['RoperDOI']
    if roper_id in done_ids:
        continue

    url = urljoin(BASE_URL, roper_id)
    study_date_raw = None
    success = False

    for attempt in range(MAX_RETRIES):
        try:
            driver.get(url)
            time.sleep(WAIT_SECONDS)
    
            # Get all divs
            print(f"Processing {roper_id} -> {url}")
            divs = driver.find_elements('tag name', 'div')
            found = False
            for i in range(len(divs)-1):
                text = divs[i].text.strip()
                if text == "Study Date:":
                    found = True
                    date_text = divs[i+1].text.strip()
                    print("✅ Found Study Date, value:", date_text)
                    study_date_raw = date_text
                    break
            if not found:
                print("❌ 'Study Date:' label not found.")
    
            success = True
            break  # break retry loop
    
        except (TimeoutException, WebDriverException, NoSuchElementException) as e:
            print(f"⚠️ Error during attempt {attempt+1} for {url}: {e}")
            time.sleep(3)  # wait before retry


    # Store result or error
    new_row = row.to_dict()
    new_row['study_date_raw'] = study_date_raw

    if success and study_date_raw:
        results.append(new_row)
    else:
        error_entry = {"RoperDOI": roper_id, "url": url, "error": "Failed after retries"}
        errors.append(error_entry)

    # Save progress periodically
    if len(results) % SAVE_EVERY == 0:
        pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)
        pd.DataFrame(errors).to_csv(ERROR_FILE, index=False)

# --- Final Save ---
pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)
pd.DataFrame(errors).to_csv(ERROR_FILE, index=False)

driver.quit()
print("Scraping complete.")

  0%|          | 0/767 [00:00<?, ?it/s]

Processing https://doi.org/10.25940/ROPER-31086422 -> https://doi.org/10.25940/ROPER-31086422


  0%|          | 1/767 [00:06<1:24:22,  6.61s/it]

✅ Found Study Date, value: April 8, 1992 - April 9, 1992
Processing https://doi.org/10.25940/ROPER-31086629 -> https://doi.org/10.25940/ROPER-31086629


  0%|          | 2/767 [00:11<1:12:25,  5.68s/it]

...

100%|█████████▉| 766/767 [1:37:05<00:07,  7.33s/it]

✅ Found Study Date, value: July 12, 2020 - July 15, 2020
Processing https://doi.org/10.25940/ROPER-31117584 -> https://doi.org/10.25940/ROPER-31117584
⚠️ Error during attempt 1 for https://doi.org/10.25940/ROPER-31117584: Message: The element with the reference 74db24b8-dd38-45c5-9fdd-4481ed910abc is stale; either its node document is not the active document, or it is no longer connected to the DOM; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#staleelementreferenceexception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:199:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:796:5
getKnownElement@chrome://remote/content/marionette/json.sys.mjs:405:11
deserializeJSON@chrome://remote/content/marionette/json.sys.mjs:263:20
cloneObject@chrome://remote/content/marionette/json.sys.mjs:

100%|██████████| 767/767 [1:37:18<00:00,  7.61s/it]

✅ Found Study Date, value: July 14, 2020 - July 19, 2020


Scraping complete.


In [18]:
# Load scraped data
df_scraped = pd.read_csv("roper_scraped_corrected.csv")

# Extract start date from the 'study_date_raw'
def parse_start_date(raw):
    try:
        start = raw.split(" - ")[0].strip()
        return pd.to_datetime(start)
    except:
        return pd.NaT

df_scraped['start_date'] = df_scraped['study_date_raw'].apply(parse_start_date)
df_scraped['quarter'] = 'Q' + df_scraped['start_date'].dt.quarter.astype(str)

# Save final dataset
df_scraped.to_csv("roper_scraped_final.csv", index=False)
